### **Group Project**

#### **Imports**

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### **Reading the data**

In [51]:
songs_df = pd.read_csv('tcc_ceds_music.csv')
songs_df.drop(songs_df.columns[0], axis=1, inplace=True)

In [52]:
songs_df

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,0.001284,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,0.001350,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,0.001284,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,0.001504,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [53]:
songs_df.columns

Index(['artist_name', 'track_name', 'release_date', 'genre', 'lyrics', 'len',
       'dating', 'violence', 'world/life', 'night/time', 'shake the audience',
       'family/gospel', 'romantic', 'communication', 'obscene', 'music',
       'movement/places', 'light/visual perceptions', 'family/spiritual',
       'like/girls', 'sadness', 'feelings', 'danceability', 'loudness',
       'acousticness', 'instrumentalness', 'valence', 'energy', 'topic',
       'age'],
      dtype='object')

In [54]:
songs_df = songs_df[['track_name', 'lyrics']]
songs_df

,track_name,lyrics
0,mohabbat bhi jhoothi,hold time feel break feel untrue convince spea...
1,i believe,believe drop rain fall grow believe darkest ni...
2,cry,sweetheart send letter goodbye secret feel bet...
3,patricia,kiss lips want stroll charm mambo chacha merin...
4,apopse eida oneiro,till darling till matter know till dream live ...
...,...,...
28367,10 million ways,cause fuck leave scar tick tock clock come kno...
28368,ante up (robbin hoodz theory),minks things chain ring braclets yap fame come...
28369,whutcha want?,get ban get ban stick crack relax plan attack ...
28370,switch,check check yeah yeah hear thing call switch g...


In [55]:
books_df = pd.read_csv('google_books_1299.csv')
books_df.drop(books_df.columns[0], axis=1, inplace=True)
books_df

,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192,none,9781612626864,English,"Jul 31, 2014"
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9780758272799,English,"Jul 1, 2007"
2,The Art of Super Mario Odyssey,Nintendo,3.9,9,133.85,SAR,Take a globetrotting journey all over the worl...,Dark Horse Comics,368,"Games &amp, Activities , Video &amp, Electronic",9781506713816,English,"Nov 5, 2019"
3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320,none,9781617734076,English,"Mar 1, 2009"
4,"The Painted Man (The Demon Cycle, Book 1)",Peter V. Brett,4.5,577,28.54,SAR,The stunning debut fantasy novel from author P...,HarperCollins UK,544,"Fiction , Fantasy , Dark Fantasy",9780007287758,English,"Jan 8, 2009"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,Twas The Nightshift Before Christmas: Festive ...,Adam Kay,4.7,47,41.82,SAR,A short gift book of festive hospital diaries ...,Pan Macmillan,112,"Medical , Health Care Delivery",9781529018592,English,"Oct 17, 2019"
1295,Why We Sleep: The New Science of Sleep and Dreams,Matthew Walker,4.8,52,46.85,SAR,'Astonishing ... an amazing book ... absolutel...,Penguin UK,368,"Psychology , Cognitive Psychology &amp, Cognition",9780141983776,English,"Sep 28, 2017"
1296,How to Understand Business Finance: Edition 2,Bob Cinnamon,3.5,4,46.85,SAR,The modern marketplace is increasingly unpredi...,Kogan Page Publishers,176,none,9780749460211,English,"Apr 3, 2010"
1297,Spider-Man: Kraven's Last Hunt,J. M. DeMatteis,4.6,74,43.28,SAR,"Collects Web of Spider-Man #31-32, Amazing Spi...",Marvel Entertainment,168,none,9781302377366,English,"Dec 10, 2014"


In [56]:
books_df = books_df[['title', 'description']]
books_df

,title,description
0,Attack on Titan: Volume 13,NO SAFE PLACE LEFT At great cost to the Garris...
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Determined to make a new start in her quaint h...
2,The Art of Super Mario Odyssey,Take a globetrotting journey all over the worl...
3,Getting Away Is Deadly: An Ellie Avery Mystery,"With swollen feet and swelling belly, pregnant..."
4,"The Painted Man (The Demon Cycle, Book 1)",The stunning debut fantasy novel from author P...
...,...,...
1294,Twas The Nightshift Before Christmas: Festive ...,A short gift book of festive hospital diaries ...
1295,Why We Sleep: The New Science of Sleep and Dreams,'Astonishing ... an amazing book ... absolutel...
1296,How to Understand Business Finance: Edition 2,The modern marketplace is increasingly unpredi...
1297,Spider-Man: Kraven's Last Hunt,"Collects Web of Spider-Man #31-32, Amazing Spi..."


In [57]:
# Remove punctuation from the lyrics column
songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Remove digits from the lyrics column
songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))

# Convert all lyrics to lowercase
songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: x.lower())

# Remove stopwords from the lyrics column
stopwords = nltk.corpus.stopwords.words('english')
songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

# Lemmatize the lyrics column
lemmatizer = WordNetLemmatizer()
songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

songs_df

C:\Users\khush\AppData\Local\Temp\ipykernel_13780\4258891530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
C:\Users\khush\AppData\Local\Temp\ipykernel_13780\4258891530.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs_df['lyrics'] = songs_df['lyrics'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))
C:\Users\khush\AppData\Local\Temp\ipykernel_13780\4258891530.py:8: 

,track_name,lyrics
0,mohabbat bhi jhoothi,hold time feel break feel untrue convince spea...
1,i believe,believe drop rain fall grow believe darkest ni...
2,cry,sweetheart send letter goodbye secret feel bet...
3,patricia,kiss lip want stroll charm mambo chacha mering...
4,apopse eida oneiro,till darling till matter know till dream live ...
...,...,...
28367,10 million ways,cause fuck leave scar tick tock clock come kno...
28368,ante up (robbin hoodz theory),mink thing chain ring braclets yap fame come f...
28369,whutcha want?,get ban get ban stick crack relax plan attack ...
28370,switch,check check yeah yeah hear thing call switch g...
